In [85]:
import yaml, requests
from weakref import ref as _wref

def yload(fp):
    if isinstance(fp, dict):
        return fp  # yaml already parsed
    if isinstance(fp, str):
        fp = open(fp, 'r')
    try:
        return yaml.safe_load(fp)
    finally:
        fp.close()

In [10]:
class SwaggerHandler():
    def __init__(self):
        self.y = {}
    
    def handle_method(self, path, pk, pv):
        for pmk, pmv in pv.get('parameters', {}).items():
            self.handle_parameter(path, pk, pmk, pmv)
    
    def handle_path(self, path, methods):
        for pk, pv in methods.items():
            self.handle_method(path, pk, pv)
    
    def parse_yaml(y):
        y = yload(y)
        self.y = y
        for k,v in y['paths'].items():
            self.path = k
            self.handle_path(v)

In [156]:
def getparams(paramslist):
    rv = []
    for d in paramslist:
        if len(d) > 1:
            raise ValueError("got multiple params for single value")
        rv.append(next(iter(d.items())))
    return rv

def getparams2(paramslist):
    # assumes only one k/v pair per dict
    rv = []
    for d in paramslist:
        rv.append(next(iter(d.values())))
    return rv

def checkforref(params):
    yup = 0
    for pk, pv in params:
        if pk != '$ref':
            print(pk, pv)
        else:
            yup += 1
    if yup == len(params):
        print("all had $ref")

def parse_yaml(y):
    y = yload(y)
    for path, methods in y.get('paths', {}).items():
        for method, propval in methods.items():
            if method.startswith("x-"): continue
            params = propval.get('parameters', [])
            ps = getparams(params)
            params = ", ".join("%r=%r" % p for p in ps)
            #print("%s: %s (params=%s)" % (method.upper(), path, params))
    
class ParentNotFoundError(Exception):
    def __init__(self, msg):
        msg = "Error: weakref expired: %r"%msg
        super().__init__(msg)
        
        
class AWeakrefMixin():
    def __init__(self, ctx, parent=None):
        self._ctx = None
        self._parent = None
        
        self.ctx = ctx
        self.parent = parent
    
    @property
    def ctx(self):
        return self._ctx()
    
    @ctx.setter
    def ctx(self, ctx):
        self._ctx = _wref(ctx)
    
    @property
    def parent(self):
        return self._parent()
    
    @parent.setter
    def parent(self, parent):
        # can't make weakref to NoneType, 
        # so use a dummy callable
        if parent is None:
            def parent():
                return None
            self._parent = parent
        else:
            self._parent = _wref(parent)
    
    
class APath(AWeakrefMixin):
    def __init__(self, ctx, name, methods, params):
        self.url = name
        self.attrib = {}
        self.methods = []
        super().__init__(ctx, ctx)
        self.extract_methods(methods, params)
        
    def extract_methods(self, methods, params):
        for meth, properties in methods.items():
            if meth.startswith("x-"):
                self.attrib[meth] = properties
                continue
            else:
                method = AMethod(self.ctx, self, meth, properties, params)
                self.methods.append(method)
                
    
class AMethod(AWeakrefMixin):
    _allowed_methods = {
        "GET",
        "PUT",
        "POST",
        "DELETE"
    }
    
    def __init__(self, ctx, parent, meth, props, params):
        super().__init__(ctx, parent)
        meth = meth.upper()
        if meth not in self._allowed_methods:
            raise ValueError(meth)
        self.summary = props.get('summary', "")
        self.description = props.get('description', "")
        self.func_name = props['operationId']
        self.params = []
        self.extract_params(props, params)
        
    def extract_params(self, props, params=None):
        # pparams is a dict of stripped name -> AParameter
        # mparams is a list[($ref)]
        pparams = params or {}
        mparams = getparams2(props.get('parameters', []))
        
        self.params.extend(pparams.values())
        self.params.extend(_resolve_ref(self.ctx.parameters, ref) for ref in mparams)
        print(self.params)
        
        
def _resolve_ref(pmap, ref):
    return pmap[ref.replace(prefix, "")]
            
class AParameter(AWeakrefMixin):
    _primitives = {
        'string',
        'integer'
    }
    def __init__(self, ctx, name, properties):
        super().__init__(ctx, None)
        self.name = properties['name']
        self.loc = properties['in']
        self.required = properties.get('required', False)
        self.schema = ASchema(self.ctx, properties['name'], properties['schema'])
       
    
def _type_to_param(ctx, schema):
    t = None
    if 'allOf' in schema or \
        'oneOf' in schema or \
        'anyOf' in schema:
            Param = ComboParam
    else:
        if "$ref" in schema:
            path = schema['$ref']
            try:
                return ctx.schemas[path]
            except KeyError:
                return UnresolvedRefParam(path, ctx)
        else:
            t = schema.get('type', None)
            if (t is None or t == 'object') and 'properties' in schema:
                Param = ObjectParam
            elif 'enum' in schema:
                Param = EnumParam
            elif t in ('string', 'integer', 'boolean'):
                Param = BasicParam
            elif t == 'number':
                Param = NumberParam
            elif t == 'array':
                Param = ArrayParam
            else:
                raise ValueError("%s: %s"%(t, schema))

        return Param(ctx, t, schema)
    
    
    
class BasicParam():
    def __init__(self, ctx, type, schema):
        self.type = type
        self.has_default = 'default' in schema
        if self.has_default:
            self.default = schema['default']
        else:
            self.default = None
        self.minimum = schema.get('minimum', None)  # numbers only
        self.schema = schema
        
class NumberParam(BasicParam):
    def __init__(self, ctx, type, schema):
        super().__init__(ctx, type, schema)
        
        
class EnumParam(BasicParam):
    def __init__(self, ctx, type, schema):
        super().__init__(ctx, type, schema)
        self.allowed = schema['enum']
    
   
class HasContentsMixin():
    def __init__(self):
        self.contents = []
        
    def __getitem__(self, index):
        return self.contents[index]
    
    @property
    def length(self):
        return len(self.contents)
    

class ArrayParam(HasContentsMixin):
    def __init__(self, ctx, type, schema):
        super().__init__()
        self.type = 'array'
        self.contents.append(_type_to_param(ctx, schema['items']))
    
    
class ObjectParam(HasContentsMixin):
    def __init__(self, ctx, name, schema):
        super().__init__()
        self.type = 'object'
        self.name = name
        self.description = schema.get('description', "")
        for k, v in schema['properties'].items():
            property = _type_to_param(ctx, v)
            self.contents.append((k, property))       
            
            
class ComboParam(HasContentsMixin):
    def __init__(self, ctx, name, schema):
        super().__init__()
        if len(schema) > 1:
            print("Don't know how to handle %s" % schema)
        op, values = next(iter(schema.items()))
        for sma in values:
            p = _type_to_param(ctx, sma)
            self.contents.append(p)

            
class UnresolvedRefParam():
    def __init__(self, path, ctx):
        self.path = path
        self.ctx = ctx
        
    def resolve(self):
        try:
            return self.ctx.schemas[self.path]
        except KeyError:
            raise
    
def iscontainer(p):
    return hasattr(p, "__getitem__")
    
def _fix(sub, i, par):
    if isinstance(sub, UnresolvedRefParam):
        par[i] = sub.resolve()
    
def resolve(ctx, param):
    if iscontainer(param):
        for i in range(param.length):
            sp = param[i]
            _fix(sp, i, param)
            resolve(ctx, sp)
    else:
        assert not isinstance(param, UnresolvedRefParam)
        
    
class ASchema(AWeakrefMixin):
    def __init__(self, ctx, name, properties):
        super().__init__(ctx)
        self.name = name
        self.param = _type_to_param(self.ctx, properties)
        self.description = properties.get('description', "")
        
    def iter(self):
        return self.param.iter()
    
    def resolve_unrefs(self):
        if isinstance(self.param, UnresolvedRefParam):
            self.param = self.param.resolve()
        else:
            self.param.resolve()

        
class Context():
    def __init__(self):
        self.parameters = {}
        self.schemas = {}
    
    
def extract_params(ctx, pmap, path, params):
    for k,v in params.items():
        parameter = AParameter(ctx, k, v)
        thepath = path + k
        pmap[thepath] = parameter
                
            
def extract_schemas(ctx, smap, path, params):
    for k,v in params.items():
        schema = ASchema(ctx, k, v)
        thepath = path + k
        smap[thepath] = schema
    
def parse_yaml2(y):
    y = yload(y)
    ctx = Context()
    
    extract_schemas(ctx, ctx.schemas, "#/components/schemas/",
                   y['components']['schemas'])
    extract_params(ctx, ctx.parameters, "#/components/parameters/",
                   y['components']['parameters'])
    print(ctx.schemas.keys())
    paths = []
    for path, methods in y.get('paths', {}).items():
        pparams = {}
        extract_params(ctx, ctx.parameters, "", methods.get('parameters', {}))
        if pparams:
            print(pparams)
        ctx.parameters.update(pparams)
        path = APath(ctx, path, methods, pparams)
        paths.add(path)
            

In [157]:
parse_yaml2(y)

ValueError: object: {'nullable': True, 'type': 'object'}

In [20]:
thefile = "c:/users/nathan/downloads/helixalmrestapi.yaml"
y = yload(thefile)

In [22]:
y['paths']['/{projectID}/documents/{itemID}/snapshots']['get']['parameters']

[{'$ref': '#/components/parameters/projectID'},
 {'$ref': '#/components/parameters/itemID'}]